In [1]:
import os
import pandas as pd
import numpy as np
import random
from pandas import DataFrame
from datetime import datetime

workdir = "/home/sequser/ResNetIC50/MEYDG-51K"

In [2]:
# < Important note > [ Please download input data from https://mega.nz/#F!CeYGDKyS!uqkmWJ4E2XSGJp_C2VO2gg]

dataset = np.load(workdir + "/MEYDG51K_dataset.npz") # input file
ss0 = np.load(workdir + '/MEYDG51K_dataset_r2.npz') # split for training and test

In [3]:
x = dataset['x']
y = dataset['y']
# y_linear = dataset['y_lnIC50']
ss0_train = ss0['train']
ss0_test = ss0['test']

In [4]:
training_image_array, training_label_array = x[ss0_train], y[ss0_train]
test_image_array, test_label_array = x[ss0_test], y[ss0_test]

# # In[9]:
# ori = training_image_array
# bat = np.zeros((ori.shape[0],178))
# cat = np.hstack([ori,bat])
# training_image_array = cat

# # In[8]:
# training_image_array.shape

# # In[10]:
# ori2 = test_image_array
# bat2 = np.zeros((ori2.shape[0],178))
# cat2 = np.hstack([ori2,bat2])
# test_image_array = cat2


In [5]:
# In[15]:
ab =[]
for i in range(50,100):
    ab.append(len(training_image_array) % i)
    
print(min(ab), ab.index(min(ab)))

2 47


In [6]:
x.shape

(51951, 62718)

In [7]:
training_image_array.shape

(46756, 62718)

In [8]:
train_X, train_y, test_X, test_y = training_image_array, training_label_array, test_image_array, test_label_array

# if K.image_data_format() == 'channels_first':
#     train_X = train_X.reshape(train_X.shape[0], 1, img_rows, img_cols)
#     test_X = test_X.reshape(x_test.shape[0], 1, img_rows, img_cols)
#     input_shape = (1, img_rows, img_cols)
# else:
#     train_X = train_X.reshape(train_X.shape[0], img_rows, img_cols, 1)
#     test_X = test_X.reshape(test_X.shape[0], img_rows, img_cols, 1)
#     input_shape = (img_rows, img_cols, 1)


In [9]:
# train_X = train_X.reshape(train_X.shape[0], train_X.shape[1], 1)
# test_X = test_X.reshape(test_X.shape[0], test_X.shape[1], 1)
# #input_shape = (img_rows, img_cols, 1)

In [10]:
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(46756, 62718) (46756,) (5195, 62718) (5195,)


In [11]:
inds = np.where(np.isnan(test_X))

In [12]:
inds

(array([], dtype=int64), array([], dtype=int64))

In [13]:
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
# train_X /= 255
# test_X /= 255
print('train_X shape:', train_X.shape)
print(train_X.shape[0], 'train samples')
print(test_X.shape[0], 'test samples')

train_X shape: (46756, 62718)
46756 train samples
5195 test samples


In [14]:
import numpy as np
from sklearn.svm import SVR
import matplotlib.pyplot as plt

# #############################################################################
StartTime8 = datetime.now()
print("StartTime :", StartTime8)
# Fit regression model
svr_lin = SVR(C=0.01)
y_lin = svr_lin.fit(train_X, train_y).predict(test_X)

EndTime8 = datetime.now()
print("EndTime :", EndTime8)

In [15]:
import matplotlib
from matplotlib import pyplot as plt

In [17]:
import pickle
import joblib
workdir = "/home/sequser/ResNetIC50/result/MEYDG-51K/SVR"
joblib.dump(svr_lin, workdir + '//MEYDG51K_r2_SVR_model.pkl') 


['/home/sequser/ResNetIC50/result/MEYDG-51K/SVR//MEYDG51K_r2_SVR_model.pkl']

In [18]:
a = pd.DataFrame(y_lin)
b = pd.DataFrame(test_label_array)
c = pd.concat([a,b], axis=1)
c.columns=["Predicted","Test"]

In [19]:
c.to_csv(workdir + '//MEYDG51K_r2_SVR_pred_obs_table.csv')

In [20]:
c

,Predicted,Test
0,2.588695,2.238455
1,2.014088,-5.247020
2,2.512669,3.697080
3,2.615102,-2.907118
4,2.530536,2.840567
...,...,...
5190,2.704213,1.671591
5191,2.462357,-1.002100
5192,2.391492,2.013122
5193,2.438859,2.355219


In [21]:
from scipy.stats import linregress
linregress(b[0], a[0])

LinregressResult(slope=0.02291033804848325, intercept=2.372697731199387, rvalue=0.2470318653637311, pvalue=4.432785579607369e-73, stderr=0.0012470860825800028, intercept_stderr=0.0042133917093510964)

In [22]:
from sklearn.metrics import r2_score
r2_value = r2_score(b,a)
print(r2_value)

0.004295362225076982


In [23]:
rse = ((b[0]-a[0])**2).sum()
mse = rse / len(b)
print("Final rmse value is =",np.sqrt(mse))

Final rmse value is = 2.7802786429544772


In [24]:
workdir = "/home/sequser/ResNetIC50/result/Figure/MEYDG-51K"
import seaborn as sns

g = sns.jointplot(x="Test", y="Predicted", data=c, kind='reg', color='#7C7C7C',
                  joint_kws={'line_kws':{'color':'black'}})
g.savefig(workdir+ '//MEYDG51K_r2_SVR.png', dpi=300)